In [1]:
import xml.etree.ElementTree as ET
xmldoc = ET.parse('xml/03_bagatelle@20\'fur@20elise\'@20woo.59.xml')
root = xmldoc.getroot()

In [2]:
for child in root:
    print(child.tag, child.attrib)

('identification', {})
('part-list', {})
('part', {'id': 'P1'})


In [3]:
for child in root.find('part'):
    print(child.tag, child.attrib)

('measure', {'number': '1'})
('measure', {'number': '2'})
('measure', {'number': '3'})
('measure', {'number': '4'})
('measure', {'number': '5'})
('measure', {'number': '6'})
('measure', {'number': '7'})
('measure', {'number': '8'})
('measure', {'number': '9'})


In [4]:
measurelist = [x for x in root.find('part')]
print len(measurelist)

9


In [5]:
notelist = []
for x in root.find('part'):
    for y in x.findall('note'):
        notelist.append(y)
print len(notelist)

40


In [6]:
# Duration calculation
#   - Negative duration indicates rests

list_duration = []
for note in notelist:
    value = int(note.findall('.//duration')[0].text)
    if note.find('rest')!=None:
        list_duration.append( -1 * value ) # If is rest, add '-' before duration
    else:
        list_duration.append( value ) # else, just put duration

print "Duration"
print len(list_duration)
print list_duration

Duration
40
[1, 1, 1, 1, 1, 1, 1, 1, 2, -1, 1, 1, 1, 2, -1, 1, 1, 1, 2, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, -1, 1, 1, 1, 2, -1, 1, 1, 1, 4]


In [7]:
# Pitch calculation
list_pitch = []
step_list = ['C','D','E','F','G','A','B']
step_value = {'C':0,'D':2,'E':4,'F':5,'G':7,'A':9,'B':11}
shift_list = [0]*7

# Fifths, take care of global shifts
fifths = int(root.findall('.//fifths')[0].text)
if not(fifths >= -7 and fifths <= 7):
    print "[ERROR] FIFTHS ERROR"
elif fifths > 0:
    if fifths >= 1:
        shift_list[step_list.index("F")] = 1
    if fifths >= 2:
        shift_list[step_list.index("C")] = 1
    if fifths >= 3:
        shift_list[step_list.index("G")] = 1
    if fifths >= 4:
        shift_list[step_list.index("D")] = 1
    if fifths >= 5:
        shift_list[step_list.index("A")] = 1
    if fifths >= 6:
        shift_list[step_list.index("E")] = 1
    if fifths == 7:
        shift_list[step_list.index("B")] = 1
elif fifths < 0:
    if fifths <= -1:
        shift_list[step_list.index("B")] = -1
    if fifths <= -2:
        shift_list[step_list.index("E")] = -1
    if fifths <= -3:
        shift_list[step_list.index("A")] = -1
    if fifths <= -4:
        shift_list[step_list.index("D")] = -1
    if fifths <= -5:
        shift_list[step_list.index("G")] = -1
    if fifths <= -6:
        shift_list[step_list.index("C")] = -1
    if fifths == -7:
        shift_list[step_list.index("F")] = -1
        
print "Fifths: ",fifths,shift_list

# Get pitches and iterate
pitchlist = root.findall('.//pitch')
for pitch in pitchlist:
    if len(pitch.findall('.//alter')):
        alter = int(pitch.findall('.//alter')[0].text)
    else:
        alter = 0
    step = pitch.findall('.//step')[0].text
    octave = pitch.findall('octave')[0].text

    list_pitch.append(step_value[step] + int(octave)*12 + alter)

print len(list_pitch)
print list_pitch

Fifths:  0 [0, 0, 0, 0, 0, 0, 0]
35
[64, 63, 64, 63, 64, 59, 62, 60, 57, 48, 52, 57, 59, 52, 56, 59, 60, 52, 64, 63, 64, 63, 64, 59, 62, 60, 57, 48, 52, 57, 59, 52, 60, 59, 57]


In [8]:
# the OOI (Offset-Onset interval)
#   - Only for notes, not rests
list_OOI = []
OOI_sum = 0
for index,obj in enumerate(list_duration):
    if(obj < 0):
        OOI_sum += int(obj)
    else:
        list_OOI.append(-1*OOI_sum)
        OOI_sum = 0
        
# Fix the loop, pop first and add 0
list_OOI.pop(0)
list_OOI.append(0)

print "Offset-Onset interval"
print len(list_OOI)
print list_OOI

Offset-Onset interval
35
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]


In [9]:
# the IOI (Inter-Onset Interval)
list_IOI = []
IOI_sum = 0
for index,obj in enumerate(list_duration):
    if(obj > 0):
        list_IOI.append(IOI_sum)
        IOI_sum = 0
    IOI_sum += abs(obj)
    
# Fix the loop, pop first and add the last duration back
list_IOI.pop(0)
list_IOI.append(list_duration[len(list_duration)-1])

print "Inter-Onset Interval"
print len(list_IOI)
print list_IOI

Inter-Onset Interval
35
[1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 4]


In [10]:
# the register (pitch difference)
list_register = []
for index,obj in enumerate(list_pitch):
    if index != len(list_pitch)-1: # n+1 - n
        list_register.append(list_pitch[index+1]-obj)
    else: # give the last one 0
        list_register.append(0)

print "Register (pitch difference)"
print len(list_register)
print list_register

Register (pitch difference)
35
[-1, 1, -1, 1, -5, 3, -2, -3, -9, 4, 5, 2, -7, 4, 3, 1, -8, 12, -1, 1, -1, 1, -5, 3, -2, -3, -9, 4, 5, 2, -7, 8, -1, -2, 0]


In [11]:
# the difference in duration, beta
list_beta = []
for index,obj in enumerate(list_IOI):
    if index != len(list_IOI)-1:
        list_beta.append(abs(list_IOI[index+1]-obj))
    else:
        list_beta.append(0)

print "Difference in duration, beta"
print len(list_beta)
print list_beta        

Difference in duration, beta
35
[0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 3, 0]


In [12]:
# GPR2a (slur/rest)
list_GPR2a = []
for index,obj in enumerate(list_OOI):
    # Determine previous and next OOI
    if(index == 0):
        previous = 0
        next = list_OOI[index+1]
    elif(index == len(list_OOI)-1):
        previous = list_OOI[index-1]
        next = 0
    else:
        previous = list_OOI[index-1]
        next = list_OOI[index+1]
    # Calculate GPR2a
    if(previous < obj and obj > next):
        list_GPR2a.append(1)
    else:
        list_GPR2a.append(0)

print "GPR2a"
print len(list_GPR2a)
print list_GPR2a

GPR2a
35
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]


In [13]:
# GPR2b (attack point)
list_GPR2b = []
for index,obj in enumerate(list_IOI):
    if(index == 0):
        previous = 0
        next = list_IOI[index+1]
    elif(index == len(list_IOI)-1):
        previous = list_IOI[index-1]
        next = 0
    else:
        previous = list_IOI[index-1]
        next = list_IOI[index+1]
    if(previous < obj and obj > next):
        list_GPR2b.append(1)
    else:
        list_GPR2b.append(0)

print "GPR2b"
print len(list_GPR2b)
print list_GPR2b

GPR2b
35
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]


In [14]:
# GPR3a (register)
list_GPR3a = []
for index,obj in enumerate(list_register):
    if(index == 0):
        previous = 0
        next = list_register[index+1]
    elif(index == len(list_register)-1):
        previous = list_register[index-1]
        next = 0
    else:
        previous = list_register[index-1]
        next = list_register[index+1]
    if(abs(previous) < abs(obj) and abs(obj) > abs(next)):
        list_GPR3a.append(1)
    else:
        list_GPR3a.append(0)

print "GPR3a"
print len(list_GPR3a)
print list_GPR3a

GPR3a
35
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0]


In [15]:
# GPR3b and GPR3c are not included in Music XML format, not going to implement

In [16]:
# GPR3d (length)
list_GPR3d = []
for index,obj in enumerate(list_beta):
    if(index == 0):
        previous = 0
        next = list_beta[index+1]
    elif(index == len(list_beta)-1):
        previous = list_beta[index-1]
        next = 0
    else:
        previous = list_beta[index-1]
        next = list_beta[index+1]
    if(previous == 0 and obj != 0 and next == 0):
        list_GPR3d.append(1)
    else:
        list_GPR3d.append(0)

print "GPR3d"
print len(list_GPR3d)
print list_GPR3d

GPR3d
35
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [17]:
# GPR4
# P_OOI
T_GPR4 = 0
list_P_OOI = []
for index,obj in enumerate(list_OOI):
    # Determine previous and next OOI
    if(index == 0):
        previous = 0
        next = list_OOI[index+1]
    elif(index == len(list_OOI)-1):
        previous = list_OOI[index-1]
        next = 0
    else:
        previous = list_OOI[index-1]
        next = list_OOI[index+1]
    # Calculate GPR2a
    if((previous + obj + next)>0):
        list_P_OOI.append(obj/(previous + obj + next))
    else:
        list_P_OOI.append(0)
print list_P_OOI
# P_IOI
list_P_IOI = []
for index,obj in enumerate(list_IOI):
    if(index == 0):
        previous = 0
        next = list_IOI[index+1]
    elif(index == len(list_IOI)-1):
        previous = list_IOI[index-1]
        next = 0
    else:
        previous = list_IOI[index-1]
        next = list_IOI[index+1]
    list_P_IOI.append(obj/(previous + obj + next))
print list_P_IOI
# P_register
list_P_register = []
for index,obj in enumerate(list_register):
    if(index == 0):
        previous = 0
        next = list_register[index+1]
    elif(index == len(list_register)-1):
        previous = list_register[index-1]
        next = 0
    else:
        previous = list_register[index-1]
        next = list_register[index+1]
    if((abs(previous) + abs(obj) + abs(next))>0):
        list_P_register.append(abs(obj)/(abs(previous) + abs(obj) + abs(next)))
    else:
        list_P_register.append(0)
print list_P_register
# P_beta
list_P_beta = []
for index,obj in enumerate(list_beta):
    if(index == 0):
        previous = 0
        next = list_beta[index+1]
    elif(index == len(list_beta)-1):
        previous = list_beta[index-1]
        next = 0
    else:
        previous = list_beta[index-1]
        next = list_beta[index+1]
    if((previous + obj + next)>0):
        list_P_beta.append(obj/(previous + obj + next))
    else:
        list_P_beta.append(0)
print list_P_beta
# GPR4
list_GPR4 = []
for index,obj in enumerate(list_OOI):
    if(max(list_P_OOI[index],list_P_IOI[index],list_P_register[index],list_P_beta[index])>T_GPR4):
        list_GPR4.append(1)
    else:
        list_GPR4.append(0)
print list_GPR4
        
print "GPR4"
print len(list_GPR4)
print list_GPR4

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
GPR4
35
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]


In [18]:
# GPR5
# Questionable

import numpy
total_duration = sum([abs(x) for x in list_duration])
sigma=numpy.std(range(0,total_duration),ddof=0)
list_norm = [1/(sigma * numpy.sqrt(2 * numpy.pi)) * numpy.exp( - (x - total_duration/2)**2 / (2 * sigma**2) ) for x in range(0,total_duration)]
normed = [format(x/(max(list_norm)-min(list_norm))-min(list_norm/(max(list_norm)-min(list_norm))), '.2f') for x in list_norm]
print len(normed)

48


In [65]:
# function for GPR6, G(m,n,r)

Wm = 0.5
Wl = 0.5
Ws = 0.5
L = len(gpr6_temp)

# gpr6_temp is a list denoting where has a note, with 1, and where there is no note, woth 0
gpr6_temp = [0] * total_duration
nmr_sum = 0
nmr_next = 0
for y in list_duration:
    nmr_sum += nmr_next
    nmr_next = abs(y)
    if(y > 0):
        gpr6_temp[nmr_sum] = 1

def N(m,r):
    return sum(gpr6_temp[m-1:m-1+r])
        

def O(m,n,r):
#     print gpr6_temp[m-1:m-1+r]
#     print gpr6_temp[n-1:n-1+r]
    temp_out = 0
    for index,obj in enumerate(gpr6_temp[m-1:m-1+r]):
        if gpr6_temp[m-1:m-1+r][index] == 1 and gpr6_temp[n-1:n-1+r][index] == 1:
            temp_out+=2
    return temp_out

def P(m,n,r):
#     print gpr6_temp[m-1:m-1+r]
#     print gpr6_temp[n-1:n-1+r]
    temp_out = 0
    for index,obj in enumerate(gpr6_temp[m-1:m-1+r]):
        if gpr6_temp[m-1:m-1+r][index] == 1 and gpr6_temp[n-1:n-1+r][index] == 1:
            if list_pitch[sum(gpr6_temp[:m+index-1])] == list_pitch[sum(gpr6_temp[:n+index-1])]:
                temp_out += 1
    return temp_out

def G(m,n,r):
    if 1<=m and 1<=n and m<=L-r+1 and n<=L-r+1 and 1<=r and r<=L:
        return (((1-Wm)*O(m,n,r)/(N(m,r)+N(n,r)))+(Wm*P(m,n,r)/O(m,n,r)))*r**Wl
    else:
        return 0

In [66]:
# i is a note, maybe from gpr6_temp
def beat(i):
    return i

# m start with 1
def head(m,r):
    if r==0:
        return -1
    if gpr6_temp[m-1]==1:
        return m
    else:
        return head(m+1,r-1)

def tail(m,r):
    if r==0:
        return -1
    if gpr6_temp[m-2+r]==1:
        return m+r-1
    else:
        return tail(m,r-1)

In [67]:
def b(i,r):
    if i == head(beat(i),r) and i != tail(beat(i),r):
        return True
    else:
        return False

def e(i,r):
    if i != head(beat(i),r) and i == tail(beat(i),r):
        return True
    else:
        return False

def t(i,r):
    if i == head(beat(i),r) and i == tail(beat(i),r):
        return True
    else:
        return False


In [68]:
def A(i):
    out = 0
    for n in range(1,L+1):
        for r in range(1,L/2+1):
            if b(i,r) and N(n,1) >= 1:
                out += G(beat(i),n,r)*(1-Ws)
            elif e(i,r) and N(n,1) >= 1:
                out += G(beat(i)-r,n-r,r)*Ws
            elif t(i,r) and N(n,1) >= 1:
                out += G(beat(i),n,r)*(1-Ws) + G(beat(i)-r,n-r,r)*Ws
            else:
                out += 0
    return out

In [73]:

print max([A(x) for x in range(1,N(1,L)+1)])

ZeroDivisionError: float division by zero